In [16]:
#$ Entity Masking
import json
from span_marker import SpanMarkerModel


def load_entities_mapping(file_path):
    """
    Loads entities and their translations from the given JSON file, skipping invalid lines.
    Args:
        file_path (str): Path to the JSON file containing entity mappings.
    Returns:
        list: A list of dictionaries representing the JSON data.
    """
    with open(file_path, 'r') as jf:
        try:
            return json.load(jf)  # Load the entire JSON array
        except json.JSONDecodeError as e:
            print(f"Error loading JSON file: {e}")
            return []


def mask_entities_with_spanmarker(text, model):
    """
    Masks named entities in text using predictions from SpanMarkerModel.

    Args:
        text (Union[str, List[str]]): Input text(s). Can be a single string or a list of strings.
        model (SpanMarkerModel): Pretrained SpanMarker model for entity prediction.

    Returns:
        Union[Tuple[str, dict], List[Tuple[str, dict]]]:
            - For a single string:
                - masked_text (str): Text with entities replaced by placeholders.
                - mapping (dict): Mapping of placeholders to original entities.
            - For a list of strings:
                - List of (masked_text, mapping) tuples.
    """
    # Check if input is a single string or a list of strings
    is_single_string = isinstance(text, str)
    if is_single_string:
        text = [text]  # Convert single string to list for uniform processing

    # Perform predictions using the model
    predictions = model.predict(text)  # Handles batch processing

    results = []
    for i, sentence in enumerate(text):
        masked_text = sentence
        mapping = {}
        type_counters = {}
        offset = 0

        # If no predictions for this sentence
        if not predictions[i]:  # Empty list
            print(f"No entities found for text: {sentence}")
            results.append((masked_text, mapping))
            continue

        # Process each entity in the prediction
        for entity in predictions[i]:
            if entity['score']>0.7:
                entity_text = entity['span']
                entity_type = entity['label'].upper()
                start = entity['char_start_index']
                end = entity['char_end_index']

                # Counter for entity type
                if entity_type not in type_counters:
                    type_counters[entity_type] = 1
                else:
                    type_counters[entity_type] += 1

                # Generate a placeholder
                placeholder = f"[ENTITY_{entity_type}_{type_counters[entity_type]}]"

                # Replace entity with placeholder in the text
                start += offset
                end += offset
                masked_text = masked_text[:start] + placeholder + masked_text[end:]
                offset += len(placeholder) - len(entity_text)

                # Add to mapping
                mapping[placeholder] = {
                    "original_text": entity_text,
                    "type": entity_type,
                    "confidence": entity['score'],
                    "start": start,
                    "end": end
                }

        results.append((masked_text, mapping))

    #if text input is a single string
    return results[0] if is_single_string else results

def ensure_pad_token(model):
    if not hasattr(model.tokenizer, "pad_token") or model.tokenizer.pad_token is None:
        print("Setting pad_token manually...")
        model.tokenizer.pad_token = "[PAD]"
        model.tokenizer.pad_token_id = model.tokenizer.convert_tokens_to_ids("[PAD]")

In [17]:
#$ MT Translation

from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

# Load the M2M100 model and tokenizer
model_name = "facebook/m2m100_418M"
model = M2M100ForConditionalGeneration.from_pretrained(model_name)
tokenizer = M2M100Tokenizer.from_pretrained(model_name)

def translate_with_placeholders_m2m(masked_text, mapping, src_lang, tgt_lang):
    """
    Translates the non-entity parts of a text and re-inserts placeholders using M2M100.

    Args:
        masked_text (str): Text with placeholders for entities.
        mapping (dict): Mapping of placeholders to entity details.
        src_lang (str): Source language code.
        tgt_lang (str): Target language code.

    Returns:
        str: Translated text with placeholders intact.
    """
    tokenizer.src_lang = src_lang

    # Encode and translate the entire sentence
    encoded_text = tokenizer(masked_text, return_tensors="pt")
    generated_tokens = model.generate(
        **encoded_text,
        forced_bos_token_id=tokenizer.get_lang_id(tgt_lang),
        max_length=128,  # Ensure sufficient length for translation
        early_stopping=True
    )

    # Decode the translated sentence
    translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    return translated_text




huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [18]:
#$ Entity Reintegration

from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

def translate_entity(entity, src_lang, tgt_lang):
    """
    Translates an entity if its type requires translation, otherwise returns the original entity.

    Args:
        entity (str): The placeholder or entity to process.
        src_lang (str): Source language code.
        tgt_lang (str): Target language code.

    Returns:
        str: The translated or original entity.
    """
    # types that should not be translated
    non_translatable_types = {"PER", "INST", "MEDIA", "PLANT", "VEHI"}

    # Check if the entity contains one of the non-translatable types
    if any(ntype in entity for ntype in non_translatable_types):
        return entity 

    # Translate the entity using M2M100
    tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")
    model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
    tokenizer.src_lang = src_lang
    encoded_entity = tokenizer(entity, return_tensors="pt")
    generated_tokens = model.generate(**encoded_entity, forced_bos_token_id=tokenizer.get_lang_id(tgt_lang))
    translated_entity = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

    return translated_entity


def reintegrate_entities(translation, mapping, src_lang, tgt_lang):
    """
    Replaces placeholders in the translated text with original or translated entities.

    Args:
        translation (str): Translated text with placeholders.
        mapping (dict): Mapping of placeholders to original entities.
        src_lang (str): Source language code.
        tgt_lang (str): Target language code.

    Returns:
        str: Translation with entities reintegrated.
    """
    for placeholder, entity_info in mapping.items():
        # Get the original entity
        original_entity = entity_info["original_text"]

        # Translate the entity if needed
        translated_entity = translate_entity(original_entity, src_lang, tgt_lang)

        # Replace placeholder with the translated entity
        translation = translation.replace(placeholder, translated_entity)

    return translation


In [19]:
import os
it_datapath = os.path.abspath('data/italian_test.json')

    # Step 1: Load the data
print("Loading Italian data...")
italian_data = load_entities_mapping(it_datapath)

Loading Italian data...


In [20]:
italian_data

[{'id': '123fe725',
  'source_locale': 'en',
  'target_locale': 'it',
  'source': 'Which Beyoncé album had more songs, Lemonade or Everything is Love?',
  'target': 'Quale album di Beyonce contiene più canzoni: "Lemonade" o "Everything is Love"?',
  'entities': {'Q36153': {'it': 'Beyoncé', 'en': 'Beyoncé'}},
  'from': 'mintaka'},
 {'id': 'f59c6a0b',
  'source_locale': 'en',
  'target_locale': 'it',
  'source': 'Which king of England was never officially crowned?',
  'target': "Quale re d'Inghilterra non è mai stato ufficialmente incoronato?",
  'entities': {'Q21': {'it': 'Inghilterra', 'en': 'England'}},
  'from': 'mintaka'},
 {'id': '8e1db257',
  'source_locale': 'en',
  'target_locale': 'it',
  'source': 'What is the most amount of money an actor has received for one movie?',
  'target': 'A quanto corrisponde la più grande quantità di denaro guadagnata da un attore per un film?',
  'entities': {'Q33999': {'it': 'attore', 'en': 'actor'}},
  'from': 'mintaka'},
 {'id': '331a0d66',
  's

In [21]:
print("Loading SpanMarker model...")
spanmarker_model = SpanMarkerModel.from_pretrained("tomaarsen/span-marker-mbert-base-multinerd")
ensure_pad_token(spanmarker_model)

Loading SpanMarker model...
Setting pad_token manually...


In [22]:
print("Loading M2M100 translation model...")
m2m_model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
m2m_tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

Loading M2M100 translation model...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [15]:
translated_sentences = []

for idx, entry in enumerate(italian_data):  # Process only the first 10 entries

    # Get source text (English) and target text (Italian)
    eng_text = entry['source']
    itl_text = entry['target']

    # Mask entities in the English text
    masked_eng_text, mapping = mask_entities_with_spanmarker(eng_text, spanmarker_model)
    print(f"Masked English Text: {masked_eng_text}")
    print(f"Entity Mapping: {json.dumps(mapping, indent=4)}")

    # Filter entities by confidence score
    # filtered_mapping = {
    #     key: value for key, value in mapping.items() if value["confidence"] > 0.8
    # }
    # print(f"Filtered Mapping (Confidence > 0.8): {json.dumps(filtered_mapping, indent=4)}")

    # Translate non-entity parts of the text
    m2m_tokenizer.src_lang = "en"
    tgt_lang = "it"
    translated_text_with_placeholders = translate_with_placeholders_m2m(
        masked_eng_text, mapping, src_lang="en", tgt_lang="it"
    )
    print(f"Translated Text with Placeholders: {translated_text_with_placeholders}")

    # Reintegrate entities into the translated text
    final_translation = reintegrate_entities(
        translated_text_with_placeholders, mapping, src_lang="en", tgt_lang="it"
    )
    print(f"Final Translation: {final_translation}")

    # Append final translation to the results
    translated_sentences.append({
        "id": entry["id"],
        "source": eng_text,
        "target": itl_text,
        "masked_source": masked_eng_text,
        "translation": final_translation,
        "mapping": mapping
    })

# Save the results
results_path = "data/translated_italian_test_new.json"
with open(results_path, "w") as results_file:
    json.dump(translated_sentences, results_file, indent=4)

print(f"Translation results for test set saved to {results_path}")


Masked English Text: Which [ENTITY_PER_1] album had more songs, [ENTITY_MEDIA_1] or Everything is Love?
Entity Mapping: {
    "[ENTITY_PER_1]": {
        "original_text": "Beyonc\u00e9",
        "type": "PER",
        "confidence": 0.9755765199661255,
        "start": 6,
        "end": 13
    },
    "[ENTITY_MEDIA_1]": {
        "original_text": "Lemonade",
        "type": "MEDIA",
        "confidence": 0.8599872589111328,
        "start": 43,
        "end": 51
    }
}
Translated Text with Placeholders: Quale album [ENTITY_PER_1] ha più canzoni, [ENTITY_MEDIA_1] o Tutto è Amore?


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Final Translation: Quale album di Beyoncé ha più canzoni, Il limone o Tutto è Amore?
Masked English Text: Which king of [ENTITY_LOC_1] was never officially crowned?
Entity Mapping: {
    "[ENTITY_LOC_1]": {
        "original_text": "England",
        "type": "LOC",
        "confidence": 0.9962968230247498,
        "start": 14,
        "end": 21
    }
}
Translated Text with Placeholders: Quale re di [ENTITY_LOC_1] non è mai stato coronato ufficialmente?


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Final Translation: Quale re di Inghilterra non è mai stato coronato ufficialmente?
No entities found for text: What is the most amount of money an actor has received for one movie?
Masked English Text: What is the most amount of money an actor has received for one movie?
Entity Mapping: {}
Translated Text with Placeholders: Quale è la quantità di denaro che un attore ha ricevuto per un film?
Final Translation: Quale è la quantità di denaro che un attore ha ricevuto per un film?
Masked English Text: What city in the [ENTITY_LOC_1] is geographically located within [ENTITY_LOC_2] and is called the "[ENTITY_LOC_3]"?
Entity Mapping: {
    "[ENTITY_LOC_1]": {
        "original_text": "Philippines",
        "type": "LOC",
        "confidence": 0.9997709393501282,
        "start": 17,
        "end": 28
    },
    "[ENTITY_LOC_2]": {
        "original_text": "Benguet",
        "type": "LOC",
        "confidence": 0.9913216233253479,
        "start": 65,
        "end": 72
    },
    "[ENTITY_LOC

...

Final Translation: Quanti paesi nel Regno Unito?
Masked English Text: Is [ENTITY_MEDIA_1] going to be made into a TV series?
Entity Mapping: {
    "[ENTITY_MEDIA_1]": {
        "original_text": "Wheel of Time",
        "type": "MEDIA",
        "confidence": 0.9800332188606262,
        "start": 3,
        "end": 16
    }
}
Translated Text with Placeholders: [ENTITY_MEDIA_1] sarà realizzato in una serie televisiva?


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Final Translation: La ruota del tempo sarà realizzato in una serie televisiva?
Masked English Text: Which country is a part of [ENTITY_LOC_1] and was also involved in the Louisiana Purchase?
Entity Mapping: {
    "[ENTITY_LOC_1]": {
        "original_text": "North America",
        "type": "LOC",
        "confidence": 0.9998823404312134,
        "start": 27,
        "end": 40
    }
}
Translated Text with Placeholders: Quale paese fa parte di [ENTITY_LOC_1] e è stato anche coinvolto nell'acquisto di Louisiana?


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Final Translation: Quale paese fa parte di America del Nord e è stato anche coinvolto nell'acquisto di Louisiana?
Translation results for test set saved to data/translated_italian_test_new.json


In [ ]:
B

In [ ]:
# translated_sentences = []

#  # Get source text (English) and target text (Italian)
# entry = {
#         "id": "adc66bf5",
#         "source_locale": "en",
#         "target_locale": "es",
#         "source": "Which was written first, Fear and Loathing in Las Vegas or Fear and Loathing on the Campaign Trail '72?",
#         "target": "\u00bfCu\u00e1l fue escrito primer, Miedo y Asco en Las Vegas o Miedo y Asco en el sendero '72?",
#         "entities": {
#             "Q772435": {
#                 "es": "Miedo y asco en Las Vegas",
#                 "en": "Fear and Loathing in Las Vegas"
#             }
#         },
#         "from": "mintaka"
#     }
# eng_text = entry['source']
# itl_text = entry['target']

# # Mask entities in the English text
# masked_eng_text, mapping = mask_entities_with_spanmarker(eng_text, spanmarker_model)
# print(f"Masked English Text: {masked_eng_text}")
# print(f"Entity Mapping: {json.dumps(mapping, indent=4)}")

# # Filter entities by confidence score
# # filtered_mapping = {
# #     key: value for key, value in mapping.items() if value["confidence"] > 0.8
# # }
# # print(f"Filtered Mapping (Confidence > 0.8): {json.dumps(filtered_mapping, indent=4)}")

# # Translate non-entity parts of the text
# m2m_tokenizer.src_lang = "en"
# tgt_lang = "es"
# translated_text_with_placeholders = translate_with_placeholders_m2m(
#     masked_eng_text, mapping, src_lang="en", tgt_lang="es"
# )
# print(f"Translated Text with Placeholders: {translated_text_with_placeholders}")

# # Reintegrate entities into the translated text
# final_translation = reintegrate_entities(
#     translated_text_with_placeholders, mapping, src_lang="en", tgt_lang="es"
# )
# print(f"Final Translation: {final_translation}")

# # Append final translation to the results
# translated_sentences.append({
#     "id": entry["id"],
#     "source": eng_text,
#     "target": itl_text,
#     "masked_source": masked_eng_text,
#     "translation": final_translation,
#     "filtered_mapping": mapping
# })

# print(translated_sentences)

Masked English Text: Which was written first, Fear and Loathing in Las Vegas or Fear and Loathing on the Campaign Trail '72?
Entity Mapping: {}
Translated Text with Placeholders: ¿Quién fue escrito primero, El miedo y el engaño en Las Vegas o El miedo y el engaño en el camino de la campaña '72?
Final Translation: ¿Quién fue escrito primero, El miedo y el engaño en Las Vegas o El miedo y el engaño en el camino de la campaña '72?
[{'id': 'adc66bf5', 'source': "Which was written first, Fear and Loathing in Las Vegas or Fear and Loathing on the Campaign Trail '72?", 'target': "¿Cuál fue escrito primer, Miedo y Asco en Las Vegas o Miedo y Asco en el sendero '72?", 'masked_source': "Which was written first, Fear and Loathing in Las Vegas or Fear and Loathing on the Campaign Trail '72?", 'translation': "¿Quién fue escrito primero, El miedo y el engaño en Las Vegas o El miedo y el engaño en el camino de la campaña '72?", 'filtered_mapping': {}}]


In [33]:
import os
it_results_datapath = os.path.abspath('data/translated_italian_test_new.json')

    # Step 1: Load the data
print("Loading Italian data...")
italian_results = load_entities_mapping(it_results_datapath)

es_results_datapath = os.path.abspath('data/translated_spanish_test_new.json')
spanish_results = load_entities_mapping(es_results_datapath)

Loading Italian data...


In [34]:
spanish_results

[{'id': '25ed319e',
  'source': 'What is the largest park in the capital city of the UK?',
  'target': '¿Cuál es el parque más grande en la capital del Reino Unido?',
  'masked_source': 'What is the largest park in the capital city of the [ENTITY_LOC_1]?',
  'translation': '¿Cuál es el parque más grande en la ciudad capital de Reino Unido?',
  'mapping': {'[ENTITY_LOC_1]': {'original_text': 'UK',
    'type': 'LOC',
    'confidence': 0.9993182420730591,
    'start': 52,
    'end': 54}}},
 {'id': 'ff08e26c',
  'source': 'Which Twilight books did not include the character Renesmee?',
  'target': '¿Cuáles de los libros de Crepúsculo no incluyeron el personaje de Renesmee?',
  'masked_source': 'Which Twilight books did not include the character Renesmee?',
  'translation': '¿Qué libros de Twilight no incluyeron el personaje Renesmee?',
  'mapping': {}},
 {'id': '8e6811f5',
  'source': 'What is the third smallest country in Africa?',
  'target': '¿Cuál es el tercer país más pequeño de África

In [ ]:
import json
from nltk.translate.bleu_score import sentence_bleu



def calculate_bleu_scores(json_data):
    results = []
    bleu_1_total, bleu_2_total, bleu_3_total, bleu_4_total = 0, 0, 0, 0
    num_objects = len(json_data)

    for entry in json_data:
        target = entry["target"].split()
        translation = entry["translation"].split()

        # Calculate BLEU scores
        bleu_1 = sentence_bleu([target], translation, weights=(1, 0, 0, 0))
        bleu_2 = sentence_bleu([target], translation, weights=(0.5, 0.5, 0, 0))
        bleu_3 = sentence_bleu([target], translation, weights=(0.33, 0.33, 0.33, 0))
        bleu_4 = sentence_bleu([target], translation, weights=(0.25, 0.25, 0.25, 0.25))

        # Update totals
        bleu_1_total += bleu_1
        bleu_2_total += bleu_2
        bleu_3_total += bleu_3
        bleu_4_total += bleu_4

        # Store individual results
        results.append({
            "id": entry["id"],
            "bleu_1": bleu_1,
            "bleu_2": bleu_2,
            "bleu_3": bleu_3,
            "bleu_4": bleu_4
        })

    # Calculate averages
    averages = {
        "average_bleu_1": bleu_1_total / num_objects if num_objects > 0 else 0,
        "average_bleu_2": bleu_2_total / num_objects if num_objects > 0 else 0,
        "average_bleu_3": bleu_3_total / num_objects if num_objects > 0 else 0,
        "average_bleu_4": bleu_4_total / num_objects if num_objects > 0 else 0
    }

    return averages, results

# Example usage

average_bleu_scores, individual_scores = calculate_bleu_scores(italian_results)

# Print average BLEU scores
print("Average BLEU Scores:", average_bleu_scores)

Average BLEU Scores: {'average_bleu_1': 0.5686440273979358, 'average_bleu_2': 0.4274066874160915, 'average_bleu_3': 0.30959511381609484, 'average_bleu_4': 0.19101749472097063}


/Users/inaocelli/miniconda3/envs/cs375/lib/python3.11/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/inaocelli/miniconda3/envs/cs375/lib/python3.11/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/inaocelli/miniconda3/envs/cs375/lib/python3.11/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it cont

In [35]:
average_bleu_scores, individual_scores = calculate_bleu_scores(spanish_results)

# Print average BLEU scores
print("Average BLEU Scores:", average_bleu_scores)

Average BLEU Scores: {'average_bleu_1': 0.6588756122255696, 'average_bleu_2': 0.5462921370231156, 'average_bleu_3': 0.442623408231079, 'average_bleu_4': 0.3322019438792871}


/Users/inaocelli/miniconda3/envs/cs375/lib/python3.11/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/inaocelli/miniconda3/envs/cs375/lib/python3.11/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/inaocelli/miniconda3/envs/cs375/lib/python3.11/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it cont